In [70]:
import pandas as pd
import numpy as np

In [75]:
import pandas as pd
import pymssql
import lightgbm as lgb
import datetime
import matplotlib.pyplot as plt
import warnings
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.utils import shuffle
import seaborn as sns
warnings.filterwarnings("ignore")
import requests
import json
import random
from tqdm import tqdm
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [98]:
data = pd.read_excel('latest_20_05.xlsx', dtype={'film':str})

In [100]:
#SQL QUERY 
def SQL_(movie_list=[], cinema=0, halls=[], tech=-1, premiere=False):
    
        
    p1=[]
    p2=[]
    p3=[]
    p4=[]
    if len(movie_list)!=0:
        p1=f' AllFilms.longID in {movie_list} '  
    if cinema!=0:
        p1=p1+'and'+f' AllFilms.Cinema ={cinema} ' 
    if len(halls)!=0:
        p1=p1+'and'+f' AllFilms.Halls in {halls} '
    if tech!=-1:
        p1=p1+'and'+f' AllFilms.TehCode in {tech} '
    
    query="""    select
sum(AllFilms.CountID) as 'CountID',
AllFilms.movie_code,
AllFilms.longID,
AllFilms.DATA,
AllFilms.TIMER,
AllFilms.TehCode,
AllFilms.Cinema,
AllFilms.Seats,
AllFilms.sessiond,
AllFilms.Halls
from
(
select 
count((KClients._Code)) as CountID,
KMovie._Code as movie_code,
KMovie._Fld13528 as longID,
cast( dateadd(year,-2000,KVisiting._Fld13867 ) as date) as DATA,
datepart(hour,KSession._Fld13203) as TIMER,
KTechnology._EnumOrder+1 as TehCode,
KCinema._Code as  Cinema,
KAuditorium._Fld4955 as Seats,
KSession._Number as sessiond,
KAuditorium._Code as Halls
from [KIS_AccumRg13714-from2017] as KVisiting
join KIS_Reference181 as KClients  on KClients._IDRRef=KVisiting._Fld13717RRef
join  KIS_Document13058 as KSession on KVisiting._Fld13715RRef=KSession._IDRRef
join KIS_Reference265 as KMovie on KSession._Fld13219RRef=KMovie._IDRRef
left join [KIS_Enum651] as KTechnology on KSession.[_Fld13217RRef]=KTechnology.[_IDRRef]   
left join [KIS_Reference267] as KAuditorium on KSession.[_Fld13213RRef]=KAuditorium.[_IDRRef]                                   
left join [KIS_Reference222]  as KCinema on KAuditorium.[_OwnerIDRRef]=KCinema.[_IDRRef] 


                                 
group by 
KMovie._Code,
KMovie._Fld13528 ,
cast( dateadd(year,-2000,KVisiting._Fld13867 ) as date),
datepart(hour,KSession._Fld13203),
KTechnology._EnumOrder,
KCinema._Code, KAuditorium._Fld4955 ,KAuditorium._Code,KSession._Number
union all
select 
count((KClients._Code)) as CountID,
KMovie._Code as movie_code,
KMovie._Fld13528 as longID,
cast( dateadd(year,-2000,KVisiting._Fld13867 ) as date) as DATA,
datepart(hour,KSession._Fld13203) as TIMER,
KTechnology._EnumOrder+1 as TehCode,
KCinema._Code as  Cinema,
KAuditorium._Fld4955 as Seats,
KSession._Number as sessiond,
KAuditorium._Code as Halls
from [KIS_AccumRg13714-to2017] as KVisiting
join KIS_Reference181 as KClients  on KClients._IDRRef=KVisiting._Fld13717RRef
join  KIS_Document13058 as KSession on KVisiting._Fld13715RRef=KSession._IDRRef
join KIS_Reference265 as KMovie on KSession._Fld13219RRef=KMovie._IDRRef
left join [KIS_Enum651] as KTechnology on KSession.[_Fld13217RRef]=KTechnology.[_IDRRef]   
left join [KIS_Reference267] as KAuditorium on KSession.[_Fld13213RRef]=KAuditorium.[_IDRRef]                                   
left join [KIS_Reference222]  as KCinema on KAuditorium.[_OwnerIDRRef]=KCinema.[_IDRRef]   
                                  
group by 
KMovie._Code,
KMovie._Fld13528 ,
cast( dateadd(year,-2000,KVisiting._Fld13867 ) as date),
datepart(hour,KSession._Fld13203),
KTechnology._EnumOrder,
KCinema._Code, KAuditorium._Fld4955 ,KAuditorium._Code,KSession._Number
) as AllFilms
where""" 
    
    query2= """ 
group by AllFilms.movie_code,
AllFilms.longID,
AllFilms.DATA,
AllFilms.TIMER,
AllFilms.TehCode,
AllFilms.Cinema,
AllFilms.Seats,
AllFilms.sessiond,
AllFilms.Halls
""" 
    query=query+p1+query2
        
    conn = pymssql.connect(
    host='10.128.150.49',
    user=r'InsertToMovieDB',
    password='Pgz%sE9w5SR4',
    database='sms_ws', 
    charset='cp1251')
    df = pd.read_sql(query,conn)
    
    if premiere==True:
        premieres=pd.DataFrame(df.groupby(['longID'])['DATA'].min()).reset_index()
        premieres=dict(zip(premieres.longID.values ,pd.to_datetime(premieres.DATA.values, format='%Y-%m-%d')))
        return df,premieres
    
    return df

In [101]:
films =['00000000000000000000000000001592',
'00000000000000000000000000000599',
'00000000000000000000000000001361',
'00000000000000000000000000001752',
'00000000000000000000000000001803',
'00000000000000000000000000001972',
# '00000000000000000000000000002032',
# '2b554ec5b2f3a514337aa52eda2e97e8',
'00000000000000000000000000001683',
'00000000000000000000000000001780',
'00000000000000000000000000001786',
'00000000000000000000000000000794',
'00000000000000000000000000001802',
'00000000000000000000000000001953',
'00000000000000000000000000001990',
'00000000000000000000000000000601',
'00000000000000000000000000001648',
'00000000000000000000000000001689',
# '00000000000000000000000000001991',
'00000000000000000000000000001454']

recommendation = pd.read_excel('all_films_03_05_1_replace_sport_avobe_comedy.xlsx')

In [102]:
df_films, premieres=SQL_(tuple(films),cinema=908, premiere=True)

In [103]:
# for i in premieres.keys():
#     if pd.datetime.now() - premieres[i] <= np.timedelta64(7, 'D'):
#         premieres[i] = 13
        
#     elif (pd.datetime.now() - premieres[i] > np.timedelta64(7, 'D') and (pd.datetime.now() - premieres[i] <= np.timedelta64(14, 'D'))):
#         premieres[i] = 10
        
#     elif (pd.datetime.now() - premieres[i] > np.timedelta64(14, 'D') and (pd.datetime.now() - premieres[i] <= np.timedelta64(21, 'D'))):
#         premieres[i] = 7
        
#     elif (pd.datetime.now() - premieres[i] > np.timedelta64(21, 'D') and (pd.datetime.now() - premieres[i] <= np.timedelta64(28, 'D'))):
#         premieres[i] = 5
        
#     elif (pd.datetime.now() - premieres[i] > np.timedelta64(28, 'D') and (pd.datetime.now() - premieres[i] <= np.timedelta64(35, 'D'))):
#         premieres[i] = 3
        
#     elif pd.datetime.now() - premieres[i] > np.timedelta64(35, 'D'):
#         premieres[i] = 2
for i in premieres.keys():
    if pd.datetime(2019,4,17) - premieres[i] <= np.timedelta64(7, 'D'):
        premieres[i] = 6
        
    elif (pd.datetime(2019,4,17) - premieres[i] > np.timedelta64(7, 'D') and (pd.datetime(2019,4,18) - premieres[i] <= np.timedelta64(14, 'D'))):
        premieres[i] = 5
        
    elif (pd.datetime(2019,4,17) - premieres[i] > np.timedelta64(14, 'D') and (pd.datetime(2019,4,18) - premieres[i] <= np.timedelta64(21, 'D'))):
        premieres[i] = 4
        
    elif (pd.datetime(2019,4,17) - premieres[i] > np.timedelta64(21, 'D') and (pd.datetime(2019,4,18) - premieres[i] <= np.timedelta64(28, 'D'))):
        premieres[i] = 3
        
    elif (pd.datetime(2019,4,17) - premieres[i] > np.timedelta64(28, 'D') and (pd.datetime(2019,4,18) - premieres[i] <= np.timedelta64(35, 'D'))):
        premieres[i] = 2
        
    elif pd.datetime(2019,4,17) - premieres[i] > np.timedelta64(35, 'D'):
        premieres[i] = 1

In [104]:
result = pd.DataFrame()

In [105]:
dx_restrictions = {
    '00000000000000000000000000001592':3,
    '00000000000000000000000000000794':1
}

In [106]:
imax_restrictions = {
    '00000000000000000000000000001592':4,
    '00000000000000000000000000000794':2
}

In [107]:
strong_citetech_restrictions_evening = {
'00000000000000000000000000001780':1,
'00000000000000000000000000001689':2
}

In [108]:
polite_citetech_restrictions = {
'00000000000000000000000000001802':3,
'00000000000000000000000000001592':3,
'00000000000000000000000000001972':1,
'00000000000000000000000000001683':5,
'00000000000000000000000000001780':1,
'00000000000000000000000000000794':2,
'00000000000000000000000000001953':1,
'00000000000000000000000000001689':2,
'00000000000000000000000000001454':1
}

In [109]:
premieres

{'00000000000000000000000000000599': 1,
 '00000000000000000000000000000601': 1,
 '00000000000000000000000000000794': 5,
 '00000000000000000000000000001361': 3,
 '00000000000000000000000000001454': 4,
 '00000000000000000000000000001592': 6,
 '00000000000000000000000000001648': 5,
 '00000000000000000000000000001683': 6,
 '00000000000000000000000000001689': 6,
 '00000000000000000000000000001752': 1,
 '00000000000000000000000000001780': 6,
 '00000000000000000000000000001786': 6,
 '00000000000000000000000000001802': 6,
 '00000000000000000000000000001803': 2,
 '00000000000000000000000000001953': 6,
 '00000000000000000000000000001972': 6,
 '00000000000000000000000000001990': 6}

In [110]:
result = pd.DataFrame()
for _day in tqdm(range(1,8)):
    
    current_day_df = data[data.day_of_week == _day]
    
    # 4DX
    dx_restrictions_copy = dx_restrictions.copy()
    dx_counter = sum([dx_restrictions_copy[film] for film in dx_restrictions_copy.keys()])
    while dx_counter > 0:
        
        top_dx = current_day_df[(current_day_df.Halls == 210) & (current_day_df.film.isin(dx_restrictions_copy.keys()))].sort_values('predicted_crowd', ascending=False)
        for index, row in top_dx.iterrows():
            
            if dx_restrictions_copy[row['film']] != 0:
                if not current_day_df[np.logical_and(current_day_df.Halls == row['Halls'] , current_day_df.time == row['time'])].empty:
                    row['restriction'] = 1
                    result = result.append(row)
                    current_day_df = current_day_df[~np.logical_and(current_day_df.Halls == row['Halls'] , current_day_df.time == row['time'])]              
                    dx_counter -= 1
                    dx_restrictions_copy[row['film']] -= 1
    # IMAX
    
    imax_restrictions_copy = imax_restrictions.copy()
    imax_counter = sum([imax_restrictions_copy[film] for film in imax_restrictions_copy.keys()])
    while imax_counter > 0:
        top_imax = current_day_df[(current_day_df.Halls == 16) & (current_day_df.film.isin(imax_restrictions_copy.keys()))].sort_values('predicted_crowd', ascending=False)
        for index, row in top_imax.iterrows():
            if imax_restrictions_copy[row['film']] != 0:
                if not current_day_df[np.logical_and(current_day_df.Halls == row['Halls'] , current_day_df.time == row['time'])].empty:
                    row['restriction'] = 1
                    result = result.append(row)
                    current_day_df = current_day_df[~np.logical_and(current_day_df.Halls == row['Halls'], current_day_df.time == row['time'])]
                    imax_counter -= 1
                    imax_restrictions_copy[row['film']] -= 1
                
    
              
    # s_c_r_evening
    s_c_r_evening_copy = strong_citetech_restrictions_evening.copy()
    s_c_r_evening_counter = sum([s_c_r_evening_copy[film] for film in s_c_r_evening_copy.keys()])
    while s_c_r_evening_counter > 0:
        
        top_s_c_r_evening = current_day_df[(current_day_df.time.isin([5,6,7])) & (current_day_df.Halls.isin([212,213,214,218,219,220,221,211])) & (current_day_df.film.isin(s_c_r_evening_copy.keys()))].sort_values('predicted_crowd', ascending=False)
        
        for index, row in top_s_c_r_evening.iterrows():
            if s_c_r_evening_copy[row['film']] != 0:
                if not current_day_df[np.logical_and(current_day_df.Halls == row['Halls'] , current_day_df.time == row['time'])].empty:
                    row['restriction'] = 1
                    result = result.append(row)
                    current_day_df = current_day_df[~np.logical_and(current_day_df.Halls == row['Halls'], current_day_df.time == row['time'])]
                    s_c_r_evening_counter -= 1
                    s_c_r_evening_copy[row['film']] -= 1
                
                
    # p_c_r
    p_c_r_copy = polite_citetech_restrictions.copy()
    p_c_r_counter = sum([p_c_r_copy[film] for film in p_c_r_copy.keys()])
    while p_c_r_counter > 0:
        
        top_p_c_r = current_day_df[(current_day_df.Halls.isin([212,213,214,218,219,220,221,211])) & (current_day_df.film.isin(p_c_r_copy.keys()))].sort_values('predicted_crowd', ascending=False)
        
        for index, row in top_p_c_r.iterrows():
            if p_c_r_copy[row['film']] != 0:
                if not current_day_df[np.logical_and(current_day_df.Halls == row['Halls'] , current_day_df.time == row['time'])].empty:
                    row['restriction'] = 1
                    result = result.append(row)
                    current_day_df = current_day_df[~np.logical_and(current_day_df.Halls == row['Halls'], current_day_df.time == row['time'])]
                    p_c_r_counter -= 1
                    p_c_r_copy[row['film']] -= 1
                
#     #other 
#     idx = current_day_df.drop(columns=['TehCode']).groupby(['Halls', 'time'])['predicted_crowd'].transform(max) == current_day_df['predicted_crowd']
#     result = result.append(current_day_df[idx])
#     ak = current_day_df
#     break

    # other  
    f_m_c_copy = premieres.copy()
    places = current_day_df.drop_duplicates(['Halls','time'])[['Halls','time']]
    for index, place in places.iterrows():
        top_films = current_day_df[(current_day_df.Halls == place.Halls) & (current_day_df.time == place.time)].sort_values(by=['predicted_crowd'], ascending=False)
        for index, top_film in top_films.iterrows():
            if f_m_c_copy[top_film.film] != 0:
                if not current_day_df[np.logical_and(current_day_df.Halls == top_film['Halls'] , current_day_df.time == top_film['time'])].empty:
                    top_film['restriction'] = 0
                    result = result.append(top_film)
                    current_day_df = current_day_df[~np.logical_and(current_day_df.Halls == top_film['Halls'], current_day_df.time == top_film['time'])]
                    f_m_c_copy[top_film.film] -= 1
#                     break

    #other2 
    idx = current_day_df.drop(columns=['TehCode']).groupby(['Halls', 'time'])['predicted_crowd'].transform(max) == current_day_df['predicted_crowd']
    current_day_df['restriction'] = 0
    result = result.append(current_day_df[idx])
    
    
    
    
    
    
#     f_m_c_copy = premieres.copy()
#     f_m_c_counter = sum([f_m_c_copy[film] for film in f_m_c_copy.keys()])
#     while f_m_c_counter > 0:
        
#         top_f_m_c = current_day_df[current_day_df.film.isin(p_c_r_copy.keys())].sort_values('predicted_crowd', ascending=False)
        
#         for index, row in top_f_m_c.iterrows():
#             if f_m_c_copy[row['film']] != 0:
#                 if not current_day_df[np.logical_and(current_day_df.Halls == row['Halls'] , current_day_df.time == row['time'])].empty:
#                     result = result.append(row)
#                     current_day_df = current_day_df[~np.logical_and(current_day_df.Halls == row['Halls'], current_day_df.time == row['time'])]
#                     f_m_c_counter -= 1
#                     f_m_c_copy[row['film']] -= 1

                
                
                

100%|██████████| 7/7 [00:09<00:00,  1.43s/it]


In [111]:
result.shape

(490, 8)

In [113]:
result.predicted_crowd = result.predicted_crowd.astype(int)
result.to_excel('20_05_latest_with_restriction_1.xlsx', index=False)

In [114]:
result.groupby('day_of_week').sum()

,Halls,TehCode,predicted_crowd,restriction,time
day_of_week,,,,,
1.0,13678.0,589.0,1830,32.0,280.0
2.0,13678.0,591.0,2254,32.0,280.0
3.0,13678.0,594.0,3842,32.0,280.0
4.0,13678.0,597.0,3801,32.0,280.0
5.0,13678.0,588.0,1321,32.0,280.0
6.0,13678.0,587.0,1466,32.0,280.0
7.0,13678.0,585.0,1357,32.0,280.0


In [68]:
result[(result.day_of_week == 7) & (result.film == '00000000000000000000000000001802')].sort_values(['time'])

,Halls,TehCode,day_of_week,film,predicted_crowd,restriction,tech,time
4799,214.0,9.0,7.0,00000000000000000000000000001802,26,1.0,[9],5.0
4800,214.0,9.0,7.0,00000000000000000000000000001802,24,1.0,[9],6.0
4898,219.0,9.0,7.0,00000000000000000000000000001802,23,1.0,[9],6.0
4801,214.0,9.0,7.0,00000000000000000000000000001802,15,0.0,[9],7.0
4899,219.0,9.0,7.0,00000000000000000000000000001802,15,0.0,[9],7.0
4997,221.0,9.0,7.0,00000000000000000000000000001802,15,0.0,[9],7.0
5046,211.0,9.0,7.0,00000000000000000000000000001802,14,0.0,[9],7.0


In [69]:
result[(result.day_of_week == 2) & (result.Halls == 213)].sort_values(['time'])

,Halls,TehCode,day_of_week,film,predicted_crowd,restriction,tech,time
5887,213.0,9.0,2.0,00000000000000000000000000000601,23,0.0,[9],1.0
6672,213.0,9.0,2.0,00000000000000000000000000001689,23,0.0,[9],2.0
6673,213.0,9.0,2.0,00000000000000000000000000001689,35,0.0,[9],3.0
2950,213.0,9.0,2.0,00000000000000000000000000001683,28,1.0,[9],4.0
6675,213.0,9.0,2.0,00000000000000000000000000001689,44,1.0,[9],5.0
6676,213.0,9.0,2.0,00000000000000000000000000001689,43,1.0,[9],6.0
1385,213.0,9.0,2.0,00000000000000000000000000001803,29,0.0,[9],7.0
